# Forward Feature selection

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

# Step 1 - Perfom data ingestion

In [2]:
import pandas as pd
df = pd.read_csv("Cars93.csv", na_values = ["", "NA"], keep_default_na = False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [3]:
df.shape

(94, 28)

# Step 2 - Perform basic data quality checks

In [5]:
df.duplicated().sum()

np.int64(1)

In [6]:
df = df.drop_duplicates(keep = "first").reset_index(drop = True)
df.shape

(93, 28)

In [8]:
m = df.isna().sum()
m[m > 0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

# Step 3 - Seperate X and Y(Weight)

In [9]:
X = df.drop(columns = ["id", "Weight"])
Y = df["Weight"]

In [10]:
X.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,non-USA,BMW 535i


In [11]:
Y.head()

0    2705
1    3560
2    3375
3    3405
4    3640
Name: Weight, dtype: int64

In [14]:
card = df.select_dtypes(include = "object").nunique() / len(df)
card

Manufacturer       0.344086
Model              1.000000
Type               0.064516
AirBags            0.032258
DriveTrain         0.032258
Cylinders          0.064516
Man.trans.avail    0.021505
Origin             0.021505
Make               1.000000
dtype: float64

In [17]:
high_card = card[card >= 0.9]
high_card

Model    1.0
Make     1.0
dtype: float64

In [18]:
X = X.drop(columns = high_card.index)
X.head()

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
0,Acura,Small,12.9,15.9,18.8,25,31,None,Front,4,...,Yes,13.2,5,177,102,68,37,26.5,11.0,non-USA
1,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,6,...,Yes,18.0,5,195,115,71,38,30.0,15.0,non-USA
2,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,6,...,Yes,16.9,5,180,102,67,37,28.0,14.0,non-USA
3,Audi,Midsize,30.8,37.7,44.6,19,26,NaN,Front,6,...,Yes,21.1,6,193,106,70,37,31.0,17.0,non-USA
4,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,4,...,Yes,21.1,4,186,109,69,39,27.0,13.0,non-USA


# Train test split

In [20]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
xtrain.shape

(74, 24)

In [22]:
xtest.shape

(19, 24)

# Apply preprocessing on X

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [25]:
num_pipe = make_pipeline(
    SimpleImputer(strategy = "median"),
    StandardScaler()
)

In [26]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    StandardScaler()
)

In [27]:
num_cols = X.select_dtypes(include = "number").columns
num_cols

Index(['Min.Price', 'Price', 'Max.Price', 'MPG.city', 'MPG.highway',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Fuel.tank.capacity',
       'Passengers', 'Length', 'Wheelbase', 'Width', 'Turn.circle',
       'Rear.seat.room', 'Luggage.room'],
      dtype='object')

In [28]:
cat_cols = X.select_dtypes(include = "object").columns
cat_cols

Index(['Manufacturer', 'Type', 'AirBags', 'DriveTrain', 'Cylinders',
       'Man.trans.avail', 'Origin'],
      dtype='object')

In [29]:
pre = ColumnTransformer(
    [
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ]
).set_output(transform = "pandas")

In [31]:
pre.fit(xtrain)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [32]:
xtrain_pre = pre.transform(xtrain)
xtrain_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Turn.circle,num__Rear.seat.room,num__Luggage.room,cat__Manufacturer,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin
65,-0.045290,-0.042952,-0.039482,-0.926566,-1.134441,0.307021,0.205378,-0.666653,-0.427741,1.004804,...,0.626076,-0.345344,-0.066366,0.618878,1.770864,1.207755,-0.127869,0.890049,-1.319371,1.055597
15,-0.279660,-0.335700,-0.355094,-0.749850,-1.134441,1.093845,0.603669,-0.666653,-1.218799,1.004804,...,1.571579,0.786859,-0.066366,-1.181494,1.770864,1.207755,-0.127869,0.890049,-1.319371,-0.947331
68,-0.338253,-0.335700,-0.319024,0.133731,0.378147,-0.479803,-0.654093,0.006826,0.627002,-0.063961,...,0.941244,-0.021857,0.635221,0.731401,-0.146852,-0.281809,-0.127869,-0.798762,-1.319371,-0.947331
78,-0.924178,-0.879377,-0.814985,1.017312,1.701662,-0.774863,-1.178161,-0.329914,-0.258982,-1.193798,...,0.310909,-0.507087,-0.767954,1.181494,0.492387,-0.281809,-0.127869,-0.798762,0.757937,-0.947331
30,-1.193703,-1.266224,-1.265859,1.547461,0.756294,-1.364981,-1.639341,-0.329914,1.861052,-2.048810,...,-1.895265,-0.668830,-0.767954,-0.618878,0.492387,1.207755,-0.127869,-0.798762,0.757937,-0.947331


In [33]:
xtest_pre = pre.transform(xtest)
xtest_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Turn.circle,num__Rear.seat.room,num__Luggage.room,cat__Manufacturer,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin
40,-0.010135,0.030236,0.068728,0.310447,0.378147,-0.381450,0.394042,1.017044,1.238753,-0.247178,...,-0.004259,-1.477547,-2.171128,-0.393831,1.131625,-1.771373,-0.127869,-0.798762,0.757937,1.055597
22,-1.076518,-1.078028,-1.022387,1.194028,0.756294,-1.168275,-1.031422,1.353784,2.145833,-1.071653,...,-2.210433,-0.507087,-1.118747,-0.843924,0.492387,1.207755,-0.127869,-0.798762,0.757937,-0.947331
55,-0.057009,-0.042952,-0.021447,-0.749850,-0.945368,0.307021,0.289229,-0.329914,-0.058581,0.882659,...,-0.004259,-0.183601,-0.066366,0.168785,1.770864,1.207755,-2.020335,0.890049,-1.319371,1.055597
72,-1.041363,-1.098939,-1.085510,1.547461,2.268882,-1.069922,-1.408751,0.680305,1.818862,-1.071653,...,-1.264930,-0.830574,0.986015,0.956447,0.492387,1.207755,-0.127869,-0.798762,0.757937,-0.947331
0,-0.490593,-0.377522,-0.282954,0.487164,0.378147,-0.873216,-0.025212,1.858893,1.312585,-1.071653,...,-0.634594,-0.507087,-1.118747,-1.631587,0.492387,1.207755,-0.127869,-0.798762,0.757937,1.055597


# Forward feature selection

In [34]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

base_model = LinearRegression()
sel = SequentialFeatureSelector(base_model, n_features_to_select="auto", scoring="r2")
sel.fit(xtrain_pre, ytrain)

,estimator estimator: estimator instanceAn unfitted estimator.,LinearRegression()
,"n_features_to_select n_features_to_select: ""auto"", int or float, default=""auto""If `""auto""`, the behaviour depends on the `tol` parameter:- if `tol` is not `None`, then features are selected while the score change does not exceed `tol`.- otherwise, half of the features are selected.If integer, the parameter is the absolute number of features to select.If float between 0 and 1, it is the fraction of features to select... versionadded:: 1.1 The option `""auto""` was added in version 1.1... versionchanged:: 1.3 The default changed from `""warn""` to `""auto""` in 1.3.",'auto'
,"tol tol: float, default=NoneIf the score is not incremented by at least `tol` between twoconsecutive feature additions or removals, stop adding or removing.`tol` can be negative when removing features using `direction=""backward""`.`tol` is required to be strictly positive when doing forward selection.It can be useful to reduce the number of features at the cost of a smalldecrease in the score.`tol` is enabled only when `n_features_to_select` is `""auto""`... versionadded:: 1.1",None
,"direction direction: {'forward', 'backward'}, default='forward'Whether to perform forward selection or backward selection.",'forward'
,"scoring scoring: str or callable, default=NoneScoring method to use for cross-validation. Options:- str: see :ref:`scoring_string_names` for options.- callable: a scorer callable object (e.g., function) with signature ``scorer(estimator, X, y)`` that returns a single value. See :ref:`scoring_callable` for details.- `None`: the `estimator`'s :ref:`default evaluation criterion ` is used.",'r2'
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used. In all othercases, :class:`~sklearn.model_selection.KFold` is used. These splittersare instantiated with `shuffle=False` so the splits will be the sameacross calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.",5
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel. When evaluating a new feature toadd or remove, the cross-validation procedure is parallel over thefolds.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None


In [36]:
sel_cols = sel.get_feature_names_out()
print(sel_cols)

['num__MPG.city' 'num__MPG.highway' 'num__Horsepower' 'num__RPM'
 'num__Passengers' 'num__Length' 'num__Wheelbase' 'num__Width'
 'cat__Manufacturer' 'cat__Type' 'cat__DriveTrain' 'cat__Origin']


In [37]:
len(sel_cols)

12

In [39]:
xtrain_pre_sel = sel.transform(xtrain_pre)
xtrain_pre_sel[0:5]

array([[-0.92656601, -1.13444106,  0.20537791, -0.66665329,  1.83716697,
         0.38931116,  1.12460871,  1.12615525,  0.61887775,  1.7708642 ,
        -0.12786929,  1.05559733],
       [-0.74984981, -1.13444106,  0.60366942, -0.66665329,  1.83716697,
        -0.40359781,  0.83413386,  1.12615525, -1.18149388,  1.7708642 ,
        -0.12786929, -0.94733093],
       [ 0.13373118,  0.37814702, -0.65409324,  0.0068258 , -0.16210297,
         0.38931116,  0.10794673,  0.11602812,  0.73140098, -0.14685215,
        -0.12786929, -0.94733093],
       [ 1.01731216,  1.70166159, -1.17816102, -0.32991375, -0.16210297,
        -0.53574931, -0.32776554, -0.38903545,  1.18149388,  0.49238663,
        -0.12786929, -0.94733093],
       [ 1.54746076,  0.75629404, -1.63934066, -0.32991375, -1.16173794,
        -2.84840049, -2.07061463, -1.65169437, -0.61887775,  0.49238663,
        -0.12786929, -0.94733093]])

In [40]:
xtest_pre_sel = sel.transform(xtest_pre)
xtest_pre_sel[0:5]

array([[ 0.31044737,  0.37814702,  0.39404231,  1.01704444, -1.16173794,
        -0.60182506, -0.61824039,  0.11602812, -0.39383129,  1.13162541,
        -0.12786929,  1.05559733],
       [ 1.19402836,  0.75629404, -1.03142204,  1.35378399, -0.16210297,
        -0.6679008 , -0.90871524, -0.89409902, -0.8439242 ,  0.49238663,
        -0.12786929, -0.94733093],
       [-0.74984981, -0.94536755,  0.28922876, -0.32991375,  1.83716697,
         0.38931116,  0.83413386,  0.62109168,  0.16878484,  1.7708642 ,
        -2.02033482,  1.05559733],
       [ 1.54746076,  2.26888212, -1.40875084,  0.6803049 , -1.16173794,
        -0.46967356, -0.76347781, -0.89409902,  0.95644743,  0.49238663,
        -0.12786929, -0.94733093],
       [ 0.48716357,  0.37814702, -0.02521191,  1.85889331, -0.16210297,
        -0.46967356, -0.32776554, -0.38903545, -1.63158679,  0.49238663,
        -0.12786929,  1.05559733]])

# Build the Linear regression model

In [41]:
from sklearn.linear_model import LinearRegression

model1 = LinearRegression()
model1.fit(xtrain_pre_sel, ytrain)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [42]:
model1.score(xtrain_pre_sel, ytrain)

0.9672855150369575

In [43]:
model1.score(xtest_pre_sel, ytest)

0.9329345515373539